In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from loading_data import get_data
from analyze_tuning_curves import get_tuning_curves
from utils_maze import get_trial_idx, get_zones

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "exploring_swrs")

In [ ]:
import info.r063d5 as info

In [ ]:
t_start = info.task_times['phase3'].start
t_stop = info.task_times['phase3'].stop

sliced_pos = position.time_slice(t_start, t_stop)

feeder1_times = []
for feeder1 in events['feeder1']:
    if t_start < feeder1 < t_stop:
        feeder1_times.append(feeder1)

feeder2_times = []
for feeder2 in events['feeder2']:
    if t_start < feeder2 < t_stop:
        feeder2_times.append(feeder2)

path_pos = get_zones(info, sliced_pos)

trials_idx, trial_epochs = get_trial_idx(path_pos['u'].time, path_pos['shortcut'].time, path_pos['novel'].time,
                                         feeder1_times, feeder2_times, t_stop)

In [ ]:
first_shortcut = trial_epochs[trials_idx['shortcut'][0][0]].start

In [ ]:
events, position, spikes, lfp, _ = get_data(info)

# Remove interneurons
max_mean_firing = 5
interneurons = np.zeros(len(spikes), dtype=bool)
for i, spike in enumerate(spikes):
    if len(spike.time) / info.session_length >= max_mean_firing:
        interneurons[i] = True
spikes = spikes[~interneurons]

# Find SWRs for the whole session
z_thresh = 2.0
power_thresh = 3.0
merge_thresh = 0.02
min_length = 0.05
swrs = nept.detect_swr_hilbert(lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                               power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

# Restrict SWRs to those with 4 or more participating neurons
swrs = nept.find_multi_in_epochs(spikes, swrs, min_involved=4)

# Find rest epochs for entire session
rest_epochs = nept.rest_threshold(position, thresh=4., t_smooth=0.05)

In [ ]:
pad = 100.
epochs_of_interest = nept.Epoch([first_shortcut-pad, first_shortcut+pad])

In [ ]:
epochs_of_interest = epochs_of_interest.intersect(rest_epochs)

duration = np.sum(epochs_of_interest.durations) / 60.

swrs = epochs_of_interest.overlaps(swrs)
swrs = swrs[swrs.durations >= 0.05]

n_swrs = swrs.n_epochs

In [ ]:
lfp.time.min()

In [ ]:
epochs_of_interest = epochs_of_interest.intersect(rest_epochs)
epochs_of_interest.n_epochs

In [ ]:
binsize = 3.
t_bins = np.arange(lfp.time.min(), lfp.time.max()+binsize, binsize)

In [ ]:
swr_counts = np.histogram(swrs.centers, bins=t_bins)[0]

In [ ]:
isthisright = nept.AnalogSignal(swr_counts, t_bins[:-1])

In [ ]:
plt.plot(isthisright.time, isthisright.data, "g", ms=3)
plt.axvline(x=first_shortcut, color="k", linestyle='--')
plt.show()

In [ ]:
t_before = 200
t_after = 200
dt = np.median(np.diff(isthisright.time))
events = [first_shortcut]

time = np.arange(-t_before, t_after+dt, dt)

data = np.zeros((len(time), len(events)))
for i, event in enumerate(events):
    sliced = isthisright.time_slice(event-t_before, event+t_after)
    data[:,i] = np.interp(time+event, sliced.time, np.squeeze(sliced.data))

ok = nept.AnalogSignal(data, time)

In [ ]:
plt.plot(sliced.time, sliced.data)
plt.show()

In [ ]:
plt.plot(ok.time, ok.data, "b")
plt.show

In [ ]:
first_shortcut

In [ ]:
rr = nept.perievent_slice(isthisright, [first_shortcut], t_before=200, t_after=200, dt=1.)

In [ ]:
plt.plot(rr.time, rr.data)
plt.axvline(x=0, color="k", linestyle='--')
plt.show()

In [ ]:
import numpy as np
import nept

data = np.array([9., 7., 5., 3., 1.])
time = np.array([0., 1., 2., 3., 4.])

analogsignal = nept.AnalogSignal(data, time)

events = np.array([1.])
perievent_lfp = nept.perievent_slice(analogsignal, events, t_before=1., t_after=1., interp=False)